<h3>NLP : Text Classification Using FastText</h3>

In [46]:
# pip install fasttext-wheel

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for fasttext-wheel from https://files.pythonhosted.org/packages/96/58/2d1c2557cefa8d30c7e7ed182cac53cc811b4dcf265ffa64fb8e8a6287c5/fasttext_wheel-0.9.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pybind11>=2.2 from https://files.pythonhosted.org/packages/04/e8/a22d08220cb5e230007d9b0c11e429c3b19d01315d1a99dbd45e3bf97386/pybind11-2.13.5-py3-none-any.whl.metadata
  Using cached pybind11-2.13.5-py3-none-any.whl.metadata (9.5 kB)
   ---------------------------------------- 0.0/232.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.4 kB ? eta -:--:--
   ------------- ------------------------- 81.9/232.4 kB 919.0 kB/s eta 0:00:01
   ---------------------------------------- 232.4/232.4 kB 1.8 MB/s eta 0:00:00
Using cached pybind11-2.13.5-py3-none-any.whl (240 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


##### Dataset Credits: https://www.kaggle.com/datasets/saurabhshahane/ecommerce-text-classification

We have a dataset of ecommerce item description. Total 4 categories,
1. Household
2. Electronics
3. Clothing and Accessories
4. Books

The task at hand is to classify a product into one of the above 4 categories based on the product description

In [49]:
import pandas as pd

df= pd.read_csv("ecommerce_dataset.csv", names=["category", "description"], header=None)
print(df.shape)
df.head(3)

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...


**Drop NA values**

In [52]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [54]:
df.category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [56]:
df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)

In [58]:
df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

When you train a fasttext model, it expects labels to be specified with __label__ prefix. We will just create a third column in the dataframe that has __label__ as well as the product description

In [61]:
df['category'] = '__label__' + df['category'].astype(str) 
df.head(5)

# Output is the label and how it should be represented in fasttext.

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [63]:
df['category_description'] = df['category'] + ' ' + df['description']
df.head(3)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...


**Pre-procesing**
1. Remove punctuation
2. Remove extra space
3. Make the entire sentence lower case

In [66]:
import re

text = "  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi"
text = re.sub(r'[^\w\s\']',' ', text)
text = re.sub(' +', ' ', text)
text.strip().lower()

"viki's bookcase bookshelf 3 shelf shelve white hi"

In [68]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower() 

In [70]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


**Train Test Split**

In [72]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [75]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [77]:
# Generating files for fasttext.

train.to_csv("ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["category_description"], index=False, header=False)

**Train the model and evaluate performance**

In [80]:
import fasttext

model = fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

# (size, precision, recall)

(10084, 0.9677707259024196, 0.9677707259024196)

First parameter (10084) is test size. Second and third parameters are precision and recall respectively. You can see we are getting around 96% precision which is pretty good

**Now let's do prediction for few product descriptions**

In [84]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3") 
# The output is the format how fast text excepts the data.

(('__label__electronics',), array([0.99455404]))

In [86]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing_accessories',), array([1.00001001]))

In [88]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00000978]))

In [90]:
model.get_nearest_neighbors("painting")

[(0.9961716532707214, 'primer'),
 (0.9956364631652832, 'lint'),
 (0.9953070878982544, 'security'),
 (0.9945001006126404, 'inverter'),
 (0.9944353699684143, 'appliance'),
 (0.994123637676239, 'stove'),
 (0.9939452409744263, 'vacuum'),
 (0.9935550689697266, 'stabilizer'),
 (0.992997407913208, 'vegetables'),
 (0.9929895997047424, 'folding')]

In [92]:
model.get_nearest_neighbors("sony")

[(0.9987949132919312, 'stereo'),
 (0.9986393451690674, 'binoculars'),
 (0.99860680103302, 'port'),
 (0.9982211589813232, 'dvd'),
 (0.9978326559066772, 'mic'),
 (0.9975266456604004, 'smartphones'),
 (0.9971380233764648, 'gps'),
 (0.9971125721931458, 'mac'),
 (0.997048020362854, 'laptops'),
 (0.9970400333404541, 'link')]

In [94]:
model.get_nearest_neighbors("banglore")

[(0.0, 'to'),
 (0.0, 'and'),
 (0.0, 'a'),
 (0.0, 'with'),
 (0.0, 'for'),
 (0.0, 'is'),
 (0.0, '</s>'),
 (0.0, 'klipsch'),
 (0.0, 'fizzytech'),
 (0.0, 'joules')]